# Road Following - Live demo (TensorRT)

In this notebook, we will use model we trained to used Jetracer interference script for a chosen category 

# TensorRT

In [2]:
import torch
device = torch.device('cuda')

Load the TRT optimized model by executing the cell below

In [3]:
import torch
from torch2trt import TRTModule

#CATEGORIES = ['apex','bottle']
# if no Categories then
CATEGORIES = []

model_trt = TRTModule()
#model_trt.load_state_dict(torch.load('road_following_model_trt.pth'))
model_trt.load_state_dict(torch.load('best_steering_model_xy_trt.pth'))

<All keys matched successfully>

### Creating the Pre-Processing Function

We have now loaded our model, but there's a slight issue. The format that we trained our model doesnt exactly match the format of the camera. To do that, we need to do some preprocessing. This involves the following steps:

1. Convert from HWC layout to CHW layout
2. Normalize using same parameters as we did during training (our camera provides values in [0, 255] range and training loaded images in [0, 1] range so we need to scale by 255.0
3. Transfer the data from CPU memory to GPU memory
4. Add a batch dimension

In [4]:
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

Awesome! We've now defined our pre-processing function which can convert images from the camera format to the neural network input format.

Now, let's start and display our camera. You should be pretty familiar with this by now. 

In [1]:
from IPython.display import display
import ipywidgets
import traitlets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera()

In [5]:
import ipywidgets.widgets as widgets
import time

target_widget = widgets.Image(format='jpeg', width=224, height=224)
x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='horizontal', description='y')

def display_xy(camera_image):
    image = np.copy(camera_image)
    x = x_slider.value
    y = y_slider.value
    x = int(x * 224 / 2 + 112)
    y = int(y * -224 / 2 + 112)
    image = cv2.circle(image, (x, y), 8, (0, 255, 0), 3)
    image = cv2.circle(image, (112, 224), 8, (0, 0,255), 3)
    image = cv2.line(image, (x,y), (112,224), (255,0,0), 3)
    jpeg_image = bgr8_to_jpeg(image)
    return jpeg_image

#time.sleep(1)
traitlets.dlink((camera, 'value'), (target_widget, 'value'), transform=display_xy)


display(widgets.HBox([target_widget]))

We'll also create our robot instance which we'll need to drive the motors.

In [6]:
from jetbot import Robot

robot = Robot()

Now, we will define sliders to control JetBot
> Note: We have initialize the slider values for best known configurations, however these might not work for your dataset, therefore please increase or decrease the sliders according to your setup and environment

1. Speed Control (speed_gain_slider): To start your JetBot increase ``speed_gain_slider`` 
2. Steering Gain Control (steering_gain_sloder): If you see JetBot is woblling, you need to reduce ``steering_gain_slider`` till it is smooth
3. Steering Bias control (steering_bias_slider): If you see JetBot is biased towards extreme right or extreme left side of the track, you should control this slider till JetBot start following line or track in the center.  This accounts for motor biases as well as camera offsets

> Note: You should play around above mentioned sliders with lower speed to get smooth JetBot road following behavior.

In [7]:
speed_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, description='speed gain')
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.09, description='steering gain')
steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.24, description='steering kd')
steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='steering bias')

display(speed_gain_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider)

FloatSlider(value=0.0, description='speed gain', max=1.0, step=0.01)

FloatSlider(value=0.09, description='steering gain', max=1.0, step=0.01)

FloatSlider(value=0.24, description='steering kd', max=0.5, step=0.001)

FloatSlider(value=0.0, description='steering bias', max=0.3, min=-0.3, step=0.01)

Next, let's display some sliders that will let us see what JetBot is thinking.  The x and y sliders will display the predicted x, y values.

The steering slider will display our estimated steering value.  Please remember, this value isn't the actual angle of the target, but simply a value that is
nearly proportional.  When the actual angle is ``0``, this will be zero, and it will increase / decrease with the actual angle.  

In [8]:
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='horizontal', description='speed')

display(y_slider, x_slider, speed_slider, steering_slider)

FloatSlider(value=0.0, description='y', max=1.0)

FloatSlider(value=0.0, description='x', max=1.0, min=-1.0)

FloatSlider(value=0.0, description='speed', max=1.0)

FloatSlider(value=0.0, description='steering', max=1.0, min=-1.0)

In [9]:
category_widget = ipywidgets.Dropdown(options=np.array(CATEGORIES), description='category')
state_widget = ipywidgets.ToggleButtons(options=['stop', 'live'], description='state', value='stop')
prediction_widget = ipywidgets.FloatText(description='prediction')

score_widgets = []
axis_categories=['x','y']
categories_number=np.array(CATEGORIES).size
for category in np.array(CATEGORIES):
    for i in axis_categories:
        category_text=category+'---'+i
        score_widget = ipywidgets.FloatSlider(min=0.0, max=1.0, description=category_text, orientation='vertical')
        score_widgets.append(score_widget)    

live_execution_widget = ipywidgets.VBox([
    ipywidgets.HBox(score_widgets),
    prediction_widget,
    category_widget,
    state_widget
])

display(live_execution_widget)

Next, we'll create a function that will get called whenever the camera's value changes. This function will do the following steps

1. Pre-process the camera image
2. Execute the neural network
3. Compute the approximate steering value
4. Control the motors using proportional / derivative control (PD)

In [17]:
angle = 0.0
angle_last = 0.0
if not CATEGORIES:
    print("List is empty.")
    category_index=0;
else:
    category_index=CATEGORIES.index(category_widget.value)   



def execute(change):
    global angle, angle_last, category_index
    image = change['new']
    xy = model_trt(preprocess(image)).detach().float().cpu().numpy().flatten()
    x = float(xy[2 * category_index])
    y = float(xy[2 * category_index + 1] )
    #for classification can create for every category a new slider
    prediction_widget.value = xy.argmax()
    if CATEGORIES:
        for i, score in enumerate(list(xy)):
            score_widgets[i].value = score
    #than add action accordingly category
    #-------------
        
    #for steering
    max_x=camera.width
    min_x=0
    max_y=camera.height
    min_y=0
    
    x = int(max_x * (x / 2.0 + 0.5))
    y = int(max_y * (y / 2.0 + 0.5))
    x_joysticklike=((x-max_x/2.0)-min_x)/(max_x-min_x)
    y_joysticklike=((max_y-y)-min_y)/(max_y-min_y)    
    
    #---------
    x_slider.value = x_joysticklike
    y_slider.value = y_joysticklike
    
    speed_slider.value = speed_gain_slider.value
    
    angle = np.arctan2(x_joysticklike, y_joysticklike)
    pid = angle * steering_gain_slider.value + (angle - angle_last) * steering_dgain_slider.value
    angle_last = angle
    
    steering_slider.value = pid + steering_bias_slider.value
    
    robot.left_motor.value = max(min(speed_slider.value + steering_slider.value, 1.0), 0.0)
    robot.right_motor.value = max(min(speed_slider.value - steering_slider.value, 1.0), 0.0)
    
execute({'new': camera.value})

List is empty.


Cool! We've created our neural network execution function, but now we need to attach it to the camera for processing.

We accomplish that with the observe function.

>WARNING: This code will move the robot!! Please make sure your robot has clearance and it is on Lego or Track you have collected data on. The road follower should work, but the neural network is only as good as the data it's trained on!

In [18]:
camera.observe(execute, names='value')

Awesome! If your robot is plugged in it should now be generating new commands with each new camera frame. 

You can now place JetBot on  Lego or Track you have collected data on and see whether it can follow track.

If you want to stop this behavior, you can unattach this callback by executing the code below.

In [ ]:
import time

camera.unobserve(execute, names='value')

time.sleep(0.1)  # add a small sleep to make sure frames have finished processing

robot.stop()

### Conclusion
That's it for this live demo! Hopefully you had some fun seeing your JetBot moving smoothly on track follwing the road!!!

If your JetBot wasn't following road very well, try to spot where it fails. The beauty is that we can collect more data for these failure scenarios and the JetBot should get even better :)